# Chapter 21: Performance Optimization

Performance is not just a technical metric—it directly impacts user experience, SEO rankings, and conversion rates. Next.js provides powerful built-in optimizations, but achieving exceptional performance requires understanding Core Web Vitals, bundle optimization, and modern React patterns. With the introduction of the React Compiler, automatic memoization is becoming the new standard.

By the end of this chapter, you'll master optimizing Core Web Vitals (LCP, INP, CLS), reducing bundle sizes with tree-shaking and dynamic imports, leveraging `next/image` and `next/font` for asset optimization, implementing code splitting strategies, using memoization effectively, integrating the React Compiler for automatic optimization, and profiling performance with browser tools.

## 21.1 Core Web Vitals

Core Web Vitals are the essential metrics Google uses to measure user experience. Understanding and optimizing them is crucial for SEO and user retention.

### Understanding the Metrics

```typescript
// Types of Core Web Vitals

/*
LCP (Largest Contentful Paint) - Loading Performance
- Target: < 2.5 seconds
- Measures: Time until largest visible element renders
- Common causes: Large images, unoptimized fonts, blocking JS

INP (Interaction to Next Paint) - Interactivity (replaces FID)
- Target: < 200 milliseconds  
- Measures: Latency of all user interactions (click, tap, keyboard)
- Common causes: Long JavaScript tasks, main thread blocking

CLS (Cumulative Layout Shift) - Visual Stability
- Target: < 0.1
- Measures: Unexpected layout shifts during page lifecycle
- Common causes: Images without dimensions, ads/embeds, web fonts
*/

// Monitoring in your app
// app/web-vitals.tsx
'use client';

import { useReportWebVitals } from 'next/web-vitals';

export function WebVitalsReporter() {
  useReportWebVitals((metric) => {
    // Send to analytics endpoint
    fetch('/api/analytics/web-vitals', {
      method: 'POST',
      body: JSON.stringify({
        name: metric.name,        // 'CLS' | 'FCP' | 'FID' | 'INP' | 'LCP' | 'TTFB'
        value: metric.value,
        rating: metric.rating,    // 'good' | 'needs-improvement' | 'poor'
        delta: metric.delta,
        entries: metric.entries,
        id: metric.id,
        navigationType: metric.navigationType,
      }),
      keepalive: true,
    });
    
    // Log in development
    console.log(`[Web Vitals] ${metric.name}: ${metric.value}`);
  });
  
  return null;
}
```

### Optimizing LCP (Largest Contentful Paint)

Improve loading speed for the main content:

```typescript
// app/page.tsx
import Image from 'next/image';
import { Suspense } from 'react';

// Good: Static generation for fast initial load
export default function HomePage() {
  return (
    <main>
      {/* Priority loading for LCP image */}
      <div className="relative h-[600px]">
        <Image
          src="/hero-image.jpg"
          alt="Hero"
          fill
          priority        // Critical for LCP
          sizes="100vw"
          className="object-cover"
          quality={85}    // Balance quality vs size
        />
        <div className="absolute inset-0 bg-gradient-to-t from-black/60" />
      </div>
      
      {/* Defer non-critical content */}
      <Suspense fallback={<div className="h-96 animate-pulse bg-gray-200" />}>
        <ProductGrid />
      </Suspense>
    </main>
  );
}

// Avoid: Layout shifts by always providing dimensions
// Bad example - causes CLS
// <img src="/image.jpg" /> 

// Good example - explicit dimensions
// <Image src="/image.jpg" width={800} height={600} />
```

### Optimizing INP (Interaction to Next Paint)

Keep the main thread responsive:

```typescript
// components/heavy-computation.tsx
'use client';

import { useTransition, useState } from 'react';

export function FilterList({ items }: { items: string[] }) {
  const [filtered, setFiltered] = useState(items);
  const [isPending, startTransition] = useTransition();

  const handleSearch = (query: string) => {
    // Non-urgent update - won't block input
    startTransition(() => {
      const results = items.filter(item => 
        item.toLowerCase().includes(query.toLowerCase())
      );
      setFiltered(results);
    });
  };

  return (
    <div>
      <input
        type="search"
        onChange={(e) => handleSearch(e.target.value)}
        placeholder="Search..."
        className="w-full p-2 border rounded"
      />
      
      {/* Show loading state during transition */}
      {isPending && (
        <div className="text-sm text-gray-500">Updating...</div>
      )}
      
      <ul className={isPending ? 'opacity-50' : ''}>
        {filtered.map(item => (
          <li key={item}>{item}</li>
        ))}
      </ul>
    </div>
  );
}

// Heavy calculations off main thread
// hooks/use-heavy-calculation.ts
export function useHeavyCalculation(data: number[]) {
  const [result, setResult] = useState<number | null>(null);
  
  useEffect(() => {
    // Move to Web Worker if calculation is heavy
    if (data.length > 10000) {
      const worker = new Worker(
        new URL('../workers/calculator.ts', import.meta.url)
      );
      
      worker.postMessage(data);
      worker.onmessage = (e) => setResult(e.data);
      
      return () => worker.terminate();
    } else {
      // Small dataset - calculate directly
      setResult(data.reduce((a, b) => a + b, 0));
    }
  }, [data]);
  
  return result;
}
```

### Optimizing CLS (Cumulative Layout Shift)

Prevent layout shifts with proper sizing:

```typescript
// components/safe-image.tsx
import Image from 'next/image';

// Always provide width/height or fill with aspect ratio
export function SafeImage({ 
  src, 
  alt, 
  aspectRatio = '16/9' 
}: { 
  src: string; 
  alt: string; 
  aspectRatio?: string;
}) {
  return (
    <div 
      className="relative w-full" 
      style={{ aspectRatio }}
    >
      <Image
        src={src}
        alt={alt}
        fill
        className="object-cover"
        sizes="(max-width: 768px) 100vw, (max-width: 1200px) 50vw, 33vw"
      />
    </div>
  );
}

// Reserve space for dynamic content
// components/dynamic-content.tsx
export function DynamicContent({ children }: { children: React.ReactNode }) {
  return (
    <div className="min-h-[200px]">
      {/* min-height prevents layout shift when content loads */}
      {children}
    </div>
  );
}

// Font optimization to prevent FOUT/FOIT
// app/layout.tsx
import { Inter } from 'next/font/google';

const inter = Inter({ 
  subsets: ['latin'],
  display: 'swap',      // Prevents invisible text during load
  variable: '--font-inter',
});

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en" className={inter.variable}>
      <body className={`${inter.className} antialiased`}>
        {children}
      </body>
    </html>
  );
}
```

## 21.2 Bundle Optimization

Reduce JavaScript bundle size for faster downloads and parsing.

### Tree Shaking and Dead Code Elimination

Ensure only used code is bundled:

```typescript
// Good: Import specific functions
import { format, parseISO } from 'date-fns';
// Instead of: import * as dateFns from 'date-fns';

// Good: Use ES modules with sideEffects: false in package.json
// lib/utils.ts
export { formatCurrency } from './format';
export { calculateTax } from './tax';
// Bundler can tree-shake unused exports

// Bad: Barrel files with unused exports
// utils/index.ts - imports everything
export * from './date';
export * from './string';
export * from './math'; // Even if only date is used, all are bundled

// Better: Explicit imports or use 
// "sideEffects": false in package.json
```

### Dynamic Imports

Split code at logical boundaries:

```typescript
// components/lazy-chart.tsx
import dynamic from 'next/dynamic';

// Heavy chart library loaded only when needed
const HeavyChart = dynamic(
  () => import('./heavy-chart').then((mod) => mod.Chart),
  {
    ssr: false,        // Disable SSR if chart uses window
    loading: () => (
      <div className="h-96 animate-pulse bg-gray-100 rounded" />
    ),
  }
);

export function AnalyticsDashboard() {
  const [showChart, setShowChart] = useState(false);
  
  return (
    <div>
      <button onClick={() => setShowChart(true)}>
        Load Analytics
      </button>
      
      {showChart && <HeavyChart data={data} />}
    </div>
  );
}

// Route-based code splitting with dynamic imports
// app/dashboard/page.tsx
import { Suspense } from 'react';

const HeavyWidget = dynamic(() => import('@/components/heavy-widget'), {
  loading: () => <p>Loading widget...</p>,
});

export default function Dashboard() {
  return (
    <div>
      <h1>Dashboard</h1>
      <Suspense fallback={<div>Loading...</div>}>
        <HeavyWidget />
      </Suspense>
    </div>
  );
}
```

### Analyzing Bundle Size

Use the bundle analyzer to identify bloat:

```javascript
// next.config.js
const withBundleAnalyzer = require('@next/bundle-analyzer')({
  enabled: process.env.ANALYZE === 'true',
});

module.exports = withBundleAnalyzer({
  // ... other config
});

// Run: ANALYZE=true npm run build
// Opens interactive visualization of bundle size
```

### Dependency Optimization

Choose lightweight alternatives:

```typescript
// Instead of moment.js (heavy)
import moment from 'moment'; // 232KB

// Use date-fns (tree-shakeable)
import { format } from 'date-fns'; // Only ~2KB for format

// Or use native Intl.DateTimeFormat (0KB)
const formatter = new Intl.DateTimeFormat('en-US', {
  dateStyle: 'long',
});

// Instead of lodash (full)
import _ from 'lodash'; // 70KB

// Use lodash-es or specific imports
import debounce from 'lodash/debounce'; // 5KB
// Or use es-toolkit (modern, tree-shakeable)
import { debounce } from 'es-toolkit';
```

## 21.3 Image and Asset Optimization

Next.js provides automatic optimization for images, fonts, and scripts.

### Next.js Image Optimization

Leverage the Image component for automatic optimization:

```typescript
// components/gallery.tsx
import Image from 'next/image';

export function Gallery({ images }: { images: string[] }) {
  return (
    <div className="grid grid-cols-3 gap-4">
      {images.map((src, index) => (
        <div key={src} className="relative aspect-square">
          <Image
            src={src}
            alt={`Gallery image ${index + 1}`}
            fill
            sizes="(max-width: 768px) 33vw, 200px"
            className="object-cover rounded-lg"
            loading={index < 6 ? 'eager' : 'lazy'}  // Load first 6 immediately
            quality={75}  // Reduce quality slightly for faster load
          />
        </div>
      ))}
    </div>
  );
}

// Remote images with domains configured
// next.config.js
module.exports = {
  images: {
    remotePatterns: [
      {
        protocol: 'https',
        hostname: 'cdn.example.com',
        port: '',
        pathname: '/images/**',
      },
    ],
    formats: ['image/webp', 'image/avif'],  // Serve modern formats
    deviceSizes: [640, 750, 828, 1080, 1200, 1920, 2048, 3840],
    imageSizes: [16, 32, 48, 64, 96, 128, 256, 384],
  },
};

// Using placeholder for better UX
import { getPlaiceholder } from 'plaiceholder';

export async function getStaticProps() {
  const { base64, img } = await getPlaiceholder('/hero.jpg');
  
  return {
    props: {
      imageProps: {
        ...img,
        blurDataURL: base64,
        placeholder: 'blur',
      },
    },
  };
}
```

### Font Optimization

Optimize font loading with `next/font`:

```typescript
// app/layout.tsx
import { Inter, Roboto_Mono } from 'next/font/google';
import localFont from 'next/font/local';

// Google Font with automatic optimization
const inter = Inter({
  subsets: ['latin'],
  display: 'swap',
  variable: '--font-inter',
  preload: true,  // Preload critical fonts
});

// Variable font (more efficient than multiple weights)
const robotoMono = Roboto_Mono({
  subsets: ['latin'],
  display: 'swap',
  variable: '--font-roboto-mono',
});

// Local font
const myFont = localFont({
  src: [
    {
      path: './fonts/my-font-regular.woff2',
      weight: '400',
      style: 'normal',
    },
    {
      path: './fonts/my-font-bold.woff2',
      weight: '700',
      style: 'normal',
    },
  ],
  variable: '--font-my',
});

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en" className={`${inter.variable} ${robotoMono.variable}`}>
      <body className="font-sans">{children}</body>
    </html>
  );
}

// tailwind.config.ts
module.exports = {
  theme: {
    extend: {
      fontFamily: {
        sans: ['var(--font-inter)', 'system-ui', 'sans-serif'],
        mono: ['var(--font-roboto-mono)', 'monospace'],
      },
    },
  },
};
```

### Script Optimization

Control third-party script loading:

```typescript
// app/layout.tsx
import Script from 'next/script';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en">
      <body>
        {children}
        
        {/* Analytics - load after interactive */}
        <Script
          src="https://analytics.example.com/script.js"
          strategy="afterInteractive"  // Default, load after page becomes interactive
        />
        
        {/* Chat widget - lazy load */}
        <Script
          src="https://chat-widget.example.com/widget.js"
          strategy="lazyOnload"  // Load when browser is idle
        />
        
        {/* Critical - load before page becomes interactive */}
        <Script
          src="https://critical-cdn.example.com/script.js"
          strategy="beforeInteractive"  // Load in <head>
        />
      </body>
    </html>
  );
}
```

## 21.4 Code Splitting Strategies

Strategic code splitting reduces initial JavaScript payload.

### Component-Level Splitting

Split heavy components:

```typescript
// app/dashboard/page.tsx
import { Suspense } from 'react';
import dynamic from 'next/dynamic';

// Heavy components loaded on demand
const HeavyChart = dynamic(() => import('@/components/heavy-chart'), {
  ssr: false,
  loading: () => <div className="h-96 bg-gray-100 animate-pulse" />,
});

const DataTable = dynamic(() => import('@/components/data-table'), {
  loading: () => <div>Loading table...</div>,
});

export default function Dashboard() {
  return (
    <div className="space-y-8">
      {/* Critical content loads immediately */}
      <DashboardStats />
      
      {/* Heavy components loaded asynchronously */}
      <Suspense fallback={<div>Loading chart...</div>}>
        <HeavyChart />
      </Suspense>
      
      <Suspense fallback={<div>Loading table...</div>}>
        <DataTable />
      </Suspense>
    </div>
  );
}
```

### Route-Based Splitting

Automatic splitting by route in App Router:

```typescript
// Next.js App Router automatically splits by route
// Each route segment is its own chunk

// For manual control, use dynamic imports in layout
// app/layout.tsx
import { Suspense } from 'react';

// Don't load heavy footer immediately
const HeavyFooter = dynamic(() => import('@/components/footer'), {
  ssr: true,
});

export default function Layout({ children }: { children: React.ReactNode }) {
  return (
    <div>
      <header>Always load header immediately</header>
      <main>{children}</main>
      
      {/* Defer footer loading */}
      <Suspense fallback={<div className="h-20 bg-gray-100" />}>
        <HeavyFooter />
      </Suspense>
    </div>
  );
}
```

### Library Splitting

Split large libraries:

```typescript
// hooks/use-map.ts
import { useEffect, useState } from 'react';

export function useMap(containerRef: React.RefObject<HTMLElement>) {
  const [map, setMap] = useState<any>(null);
  
  useEffect(() => {
    // Dynamically import map library only when needed
    const initMap = async () => {
      const L = await import('leaflet');
      await import('leaflet/dist/leaflet.css');
      
      if (containerRef.current) {
        const instance = L.map(containerRef.current).setView([51.505, -0.09], 13);
        L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png').addTo(instance);
        setMap(instance);
      }
    };
    
    initMap();
  }, [containerRef]);
  
  return map;
}
```

## 21.5 Memoization Techniques

Prevent unnecessary re-renders with strategic memoization.

### React.memo for Components

Memoize expensive components:

```typescript
// components/expensive-list.tsx
'use client';

import { memo } from 'react';

interface Item {
  id: string;
  name: string;
  price: number;
}

// Only re-renders if items actually change
export const ExpensiveList = memo(function ExpensiveList({ 
  items,
  onSelect,
}: { 
  items: Item[];
  onSelect: (id: string) => void;
}) {
  console.log('ExpensiveList rendering');
  
  return (
    <ul className="space-y-2">
      {items.map(item => (
        <li 
          key={item.id}
          onClick={() => onSelect(item.id)}
          className="p-4 border rounded hover:bg-gray-50"
        >
          {item.name} - ${item.price}
        </li>
      ))}
    </ul>
  );
}, (prevProps, nextProps) => {
  // Custom comparison - only check if items array reference changed
  return prevProps.items === nextProps.items;
});

// Parent component
'use client';

import { useState, useCallback } from 'react';
import { ExpensiveList } from './expensive-list';

export function ProductPage() {
  const [items, setItems] = useState<Item[]>([]);
  const [filter, setFilter] = useState('');
  
  // Memoize callback to prevent ExpensiveList re-render
  const handleSelect = useCallback((id: string) => {
    console.log('Selected:', id);
  }, []);  // Empty deps - function never changes
  
  // Memoize filtered items
  const filteredItems = useMemo(() => {
    return items.filter(item => 
      item.name.toLowerCase().includes(filter.toLowerCase())
    );
  }, [items, filter]);
  
  return (
    <div>
      <input 
        value={filter}
        onChange={(e) => setFilter(e.target.value)}
        placeholder="Filter..."
      />
      <ExpensiveList items={filteredItems} onSelect={handleSelect} />
    </div>
  );
}
```

### useMemo for Expensive Calculations

Cache expensive computations:

```typescript
// components/data-processor.tsx
'use client';

import { useMemo } from 'react';

export function DataProcessor({ data }: { data: number[] }) {
  // Expensive calculation cached until data changes
  const processedData = useMemo(() => {
    console.log('Processing data...');
    return data
      .filter(n => n > 0)
      .map(n => n * 2)
      .reduce((sum, n) => sum + n, 0);
  }, [data]);  // Only recalculate when data array changes
  
  // Grouping operation
  const grouped = useMemo(() => {
    return data.reduce((acc, item) => {
      const key = Math.floor(item / 10);
      if (!acc[key]) acc[key] = [];
      acc[key].push(item);
      return acc;
    }, {} as Record<number, number[]>);
  }, [data]);
  
  return <div>Total: {processedData}</div>;
}
```

### Context Optimization

Optimize context providers to prevent unnecessary re-renders:

```typescript
// contexts/theme-context.tsx
'use client';

import { createContext, useContext, useState, useMemo, memo } from 'react';

interface ThemeContextType {
  theme: 'light' | 'dark';
  toggleTheme: () => void;
}

const ThemeContext = createContext<ThemeContextType | null>(null);

// Split context into value and updater if needed
export function ThemeProvider({ children }: { children: React.ReactNode }) {
  const [theme, setTheme] = useState<'light' | 'dark'>('light');
  
  // Memoize context value
  const value = useMemo(() => ({
    theme,
    toggleTheme: () => setTheme(t => t === 'light' ? 'dark' : 'light'),
  }), [theme]);
  
  return (
    <ThemeContext.Provider value={value}>
      {children}
    </ThemeContext.Provider>
  );
}

// Memoize children to prevent re-renders when parent updates
export const MemoizedProvider = memo(function Provider({ children }) {
  return <ThemeProvider>{children}</ThemeProvider>;
});
```

## 21.6 React Compiler Integration

The React Compiler (formerly React Forget) automatically memoizes components without manual useMemo/useCallback.

### Enabling React Compiler

Configure the experimental compiler:

```javascript
// next.config.js
module.exports = {
  experimental: {
    reactCompiler: true,  // Enable React Compiler
  },
};

// Or with specific options
const ReactCompilerConfig = {
  target: '19',  // React version
};

module.exports = {
  experimental: {
    reactCompiler: {
      ...ReactCompilerConfig,
    },
  },
};
```

### Before and After

See how React Compiler optimizes code:

```typescript
// Without React Compiler - manual memoization required
'use client';

import { useState, useMemo, useCallback } from 'react';

function ManualOptimization() {
  const [count, setCount] = useState(0);
  const [name, setName] = useState('');
  
  // Manual memoization
  const expensiveValue = useMemo(() => {
    return count * 2;
  }, [count]);
  
  const handleClick = useCallback(() => {
    setCount(c => c + 1);
  }, []);
  
  return (
    <div>
      <ExpensiveComponent value={expensiveValue} onClick={handleClick} />
      <input value={name} onChange={e => setName(e.target.value)} />
    </div>
  );
}

// With React Compiler - automatic memoization
'use client';

// No imports needed, compiler adds memoization automatically
function AutomaticOptimization() {
  const [count, setCount] = useState(0);
  const [name, setName] = useState('');
  
  // Automatically memoized
  const expensiveValue = count * 2;
  
  // Automatically memoized callback
  const handleClick = () => {
    setCount(c => c + 1);
  };
  
  return (
    <div>
      <ExpensiveComponent value={expensiveValue} onClick={handleClick} />
      <input value={name} onChange={e => setName(e.target.value)} />
    </div>
  );
}
```

### When to Keep Manual Memoization

Some cases still require manual intervention:

```typescript
// Complex dependency arrays still need attention
'use client';

function ComplexComponent({ items, filter }: { items: Item[]; filter: Filter }) {
  // React Compiler handles simple cases, but complex logic may need help
  const processed = useMemo(() => {
    // Compiler might not detect deep equality of filter object
    return items.filter(item => matchesFilter(item, filter));
  }, [items, filter]);  // Explicit deps still useful for clarity
  
  return <List items={processed} />;
}

// External refs need manual handling
'use client';

function ExternalIntegration() {
  const canvasRef = useRef<HTMLCanvasElement>(null);
  
  // Manual effect for external library integration
  useEffect(() => {
    if (!canvasRef.current) return;
    
    const ctx = canvasRef.current.getContext('2d');
    // External drawing library...
    
    return () => {
      // Cleanup
    };
  }, []);  // Empty deps intentional for external setup
  
  return <canvas ref={canvasRef} />;
}
```

## 21.7 Performance Profiling

Measure and identify performance bottlenecks.

### React DevTools Profiler

Use the Profiler API programmatically:

```typescript
// components/profiler.tsx
'use client';

import { Profiler, ReactNode } from 'react';

function onRenderCallback(
  id: string,              // Component identifier
  phase: 'mount' | 'update',  // Lifecycle phase
  actualDuration: number,  // Time spent rendering
  baseDuration: number,    // Estimated time without memoization
  startTime: number,       // When React began rendering
  commitTime: number       // When React committed changes
) {
  // Log slow renders
  if (actualDuration > 16) {  // Longer than one frame (60fps)
    console.warn(`Slow render detected: ${id}`, {
      phase,
      actualDuration,
      baseDuration,
      wastedTime: baseDuration - actualDuration,  // Time saved by memoization
    });
    
    // Send to analytics
    fetch('/api/analytics/performance', {
      method: 'POST',
      body: JSON.stringify({
        component: id,
        duration: actualDuration,
        phase,
        timestamp: Date.now(),
      }),
      keepalive: true,
    });
  }
}

export function ProfiledApp({ children }: { children: ReactNode }) {
  if (process.env.NODE_ENV === 'production') {
    return children;  // Don't profile in production
  }
  
  return (
    <Profiler id="App" onRender={onRenderCallback}>
      {children}
    </Profiler>
  );
}
```

### Lighthouse CI

Automate performance testing in CI/CD:

```yaml
# .github/workflows/lighthouse.yml
name: Lighthouse CI

on: [push]

jobs:
  lighthouse:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Build Next.js app
        run: |
          npm ci
          npm run build
          npm start &
          npx wait-on http://localhost:3000
      
      - name: Run Lighthouse CI
        run: |
          npm install -g @lhci/cli@0.12.x
          lhci autorun
        env:
          LHCI_GITHUB_APP_TOKEN: ${{ secrets.LHCI_GITHUB_APP_TOKEN }}

# lighthouserc.js
module.exports = {
  ci: {
    collect: {
      url: ['http://localhost:3000/', 'http://localhost:3000/dashboard'],
      numberOfRuns: 3,
    },
    assert: {
      assertions: {
        'categories:performance': ['warn', { minScore: 0.9 }],
        'categories:accessibility': ['error', { minScore: 0.9 }],
        'first-contentful-paint': ['warn', { maxNumericValue: 2000 }],
        'largest-contentful-paint': ['error', { maxNumericValue: 2500 }],
      },
    },
    upload: {
      target: 'temporary-public-storage',
    },
  },
};
```

### Server-Timing Headers

Add server-side performance metrics:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const start = Date.now();
  
  const response = NextResponse.next();
  
  // Add timing header
  const duration = Date.now() - start;
  response.headers.set('Server-Timing', `middleware;dur=${duration}`);
  
  return response;
}

// In app (Server Component)
export default async function Page() {
  const start = performance.now();
  
  const data = await fetchData();
  
  const duration = Math.round(performance.now() - start);
  
  return (
    <>
      {/* Add to meta for debugging */}
      <meta name="server-timing" content={`data-fetch;dur=${duration}`} />
      <PageContent data={data} />
    </>
  );
}
```

## Key Takeaways from Chapter 21

1. **Core Web Vitals**: Monitor LCP (largest visible element < 2.5s), INP (interaction latency < 200ms), and CLS (layout stability < 0.1) using `useReportWebVitals`. Optimize LCP with image priority loading and font optimization, INP with `useTransition` for non-urgent updates, and CLS by always providing image dimensions and using `font-display: swap`.

2. **Bundle Optimization**: Enable tree-shaking by using ES modules and avoiding barrel files. Use dynamic imports with `next/dynamic` for heavy components, analyze bundles with `@next/bundle-analyzer`, and prefer lightweight alternatives (date-fns vs moment, es-toolkit vs lodash) to reduce JavaScript payload.

3. **Asset Optimization**: Use `next/image` with priority for LCP images, sizes for responsive images, and quality settings for balance. Optimize fonts with `next/font` for automatic self-hosting and CSS optimization. Control third-party scripts with `next/script` strategies (beforeInteractive, afterInteractive, lazyOnload).

4. **Code Splitting**: Implement route-based splitting (automatic in App Router), component-level splitting with Suspense boundaries, and library splitting by dynamically importing heavy libraries (maps, charts) only when needed. This reduces initial bundle size and improves TTI (Time to Interactive).

5. **Memoization**: Use `React.memo` for expensive pure components with stable props, `useMemo` for expensive calculations, and `useCallback` for stable function references passed to children. With React Compiler enabled, manual memoization becomes unnecessary for many cases, but complex dependency patterns may still benefit from explicit hooks.

6. **React Compiler**: Enable the experimental React Compiler in `next.config.js` to automatically memoize components without manual `useMemo`/`useCallback`. The compiler analyzes component dependencies and inserts memoization automatically, reducing boilerplate while maintaining performance.

7. **Performance Profiling**: Use React DevTools Profiler API to detect slow renders (>16ms), implement Lighthouse CI in your pipeline to enforce performance budgets (LCP < 2.5s, CLS < 0.1), and add Server-Timing headers to measure data fetching performance. Monitor real-user metrics (RUM) via `useReportWebVitals` sent to your analytics endpoint.

## Coming Up Next

**Chapter 22: SEO & Metadata**

Now that your application is blazing fast, it's time to ensure search engines can discover and properly index your content. In Chapter 22, we'll explore the Metadata API for generating dynamic meta tags, Open Graph implementation for social sharing, structured data with JSON-LD, sitemap generation, robots.txt configuration, and canonical URL management. You'll learn how to build search-engine-optimized applications that rank well and present beautifully across all platforms.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='20. testing_strategies.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='22. seo_and_metadata.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
